**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

See [Configure Service Account](https://kubernetes.io/docs/tasks/configure-pod-container/configure-service-account/)

In [1]:
kubectl get serviceaccount default -o yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-01-28T10:00:50Z"
  name: default
  namespace: default
  resourceVersion: "416"
  uid: ebd40c5d-e9e4-4e76-a52f-eb9e0a1c9d42
secrets:
- name: default-token-jt7hx


In [2]:
kubectl patch serviceaccount default  --patch "automountServiceAccountToken: false"

serviceaccount/default patched


In [3]:
kubectl get serviceaccount default -o yaml

apiVersion: v1
automountServiceAccountToken: false
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-01-28T10:00:50Z"
  name: default
  namespace: default
  resourceVersion: "2300"
  uid: ebd40c5d-e9e4-4e76-a52f-eb9e0a1c9d42
secrets:
- name: default-token-jt7hx


# ☠️ **this can be overwritten** ☠️



In [13]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-with-token-again
  labels:
    app: nginx-with-token-again
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx-with-token-again
  template:
    metadata:
      labels:
        app: nginx-with-token-again
    spec:
      automountServiceAccountToken: true
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF

deployment.apps/nginx-with-token-again configured


In [15]:
POD=$(kubectl get pods -o jsonpath='{.items[0].metadata.name}' )
echo $POD

nginx-with-token-again-66868cff8b-7llg6


In [16]:
kubectl exec  $POD -- df

Filesystem                    1K-blocks     Used Available Use% Mounted on
overlay                        61796348 42447416  16186820  73% /
tmpfs                             65536        0     65536   0% /dev
tmpfs                           7893344        0   7893344   0% /sys/fs/cgroup
/dev/mapper/sirius--vg-docker  61796348 42447416  16186820  73% /etc/hosts
shm                               65536        0     65536   0% /dev/shm
tmpfs                           7893344       12   7893332   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs                           7893344        0   7893344   0% /proc/acpi
tmpfs                           7893344        0   7893344   0% /proc/scsi
tmpfs                           7893344        0   7893344   0% /sys/firmware


In [19]:
kubectl get pods  -o yaml | grep -i token

    generateName: nginx-with-token-again-66868cff8b-
      app: nginx-with-token-again
          f:automountServiceAccountToken: {}
    name: nginx-with-token-again-66868cff8b-7llg6
      name: nginx-with-token-again-66868cff8b
    automountServiceAccountToken: true
        name: default-token-jt7hx
    - name: default-token-jt7hx
        secretName: default-token-jt7hx


In [24]:
kubectl get pods -A -o jsonpath='{.items[*].spec.automountServiceAccountToken}'

true